In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
root = '.'

# Load data
all_units = pd.read_csv(os.path.join(root, "..", "config", "chembl_processed", "UnitStringValidations.csv"), skiprows=1)
del all_units['cumulative_prop']
del all_units['Validation Result']
del all_units['Notes']
all_units = all_units[all_units['standard_units'].isna() == False].reset_index(drop=True)

# Load curated data
standard_units_conversions_MDF = pd.read_csv(os.path.join(root, "..", "config", "manual_curation", "standard_units_conversions_MDF.csv"))
ucum_GT = pd.read_csv(os.path.join(root, "..", "config", "manual_curation", "ucum_GT.csv"))

In [7]:
# Valid units
unit_to_valid_unit = {}
for unit, valid_unit in ucum_GT[['units', "val_unit"]].values:
    unit_to_valid_unit[unit] = valid_unit

# Final units
unit_to_final_unit = {}
for unit, final_unit in ucum_GT[['units', "final_unit"]].values:
    unit_to_final_unit[unit] = final_unit

# Conversion formula
unit_to_conv_formula = {}
for unit, conv_formula in ucum_GT[['units', "transformer"]].values:
    unit_to_conv_formula[unit] = conv_formula

# ADD HERE MANUAL MAPPINGS

#          ...

# Getting valid units
all_units['valid_unit'] = [unit_to_valid_unit[i] if i in unit_to_valid_unit else np.nan for i in all_units['standard_units']]

# Getting final units
all_units['final_unit'] = [unit_to_final_unit[i] if i in unit_to_final_unit else np.nan for i in all_units['standard_units']]

# Getting conversion formula
all_units['conversion_formula'] = [unit_to_conv_formula[i] if i in unit_to_conv_formula else np.nan for i in all_units['standard_units']]

# Get proportion
total_count = all_units['count'].sum()
all_units['cumulative_prop'] = (all_units['count'].cumsum() / total_count).round(3)

In [10]:
all_units[:50]

,standard_units,count,Unit String Tested,valid_unit,final_unit,conversion_formula,cumulative_prop
0,nM,12587576,nM,nmol.L-1,umol.L-1,standard_value/1000,0.602
1,%,5200597,%,%,%,standard_value,0.851
2,ug.mL-1,952582,ug.mL-1,ug.ml-1,umol.L-1,standard_value*1000/molecular_weight,0.896
3,s-1,828026,s-1,s-1,s-1,standard_value,0.936
4,uM,257898,uM,umol,umol,standard_value,0.948
5,hr,128555,hr,h,s,standard_value*60*60,0.954
6,mm,95027,mm,mm,mm,standard_value,0.959
7,ug ml-1,77448,ug ml-1,ug.ml-1,umol.L-1,standard_value*1000/molecular_weight,0.962
8,mg.kg-1,65710,mg.kg-1,mg.kg-1,ug.mg-1,standard_value/1000,0.966
9,mL.min-1.kg-1,46177,mL.min-1.kg-1,NaN,NaN,NaN,0.968
